# Data Aggregation and Group Operations

See "Python for Data Analysis,  2nd Edition',  Wes McKinney,  Chapter 10

In [1]:
import pandas as pd
import numpy as np

Approaches to aggegrating (grouping) data

-A list or array of values the same length as the axis being grouped

-A dictionary or series giving a correspondence between the values on the axes and the group names

-A function to be called on the axis index, or the labels in the index to carry out the grouping

We'll look at examples of this

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],'key2' : ['one', 'two', 'one', 'two', 'one'],'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,0.902170,0.729726
1,a,two,0.509565,0.618821
2,b,one,-1.454717,-1.239709
3,b,two,-0.337879,0.782384
4,a,one,-1.211184,-0.111124


In [3]:
# we can create a GroupBy object,  specifying we we want the column data1 grouped by key1

my_grouped=df['data1'].groupby(df['key1'])

my_grouped

In [4]:
my_grouped.mean()

key1
a    0.066851
b   -0.896298
Name: data1, dtype: float64

In [5]:
# we would probably typically see this done in one step

df['data1'].groupby(df['key1']).mean()

key1
a    0.066851
b   -0.896298
Name: data1, dtype: float64

In [6]:
# We can do a multiple grouping here

df['data1'].groupby([df['key1'], df['key2']]).mean()

key1  key2
a     one    -0.154507
      two     0.509565
b     one    -1.454717
      two    -0.337879
Name: data1, dtype: float64

In [7]:
# this structure now has a hierarchical index, or a multi-index
my_means=df['data1'].groupby([df['key1'], df['key2']]).mean()

my_means.index

MultiIndex([('a', 'one'),
            ('a', 'two'),
            ('b', 'one'),
            ('b', 'two')],
           names=['key1', 'key2'])

In [8]:
# we can show this in unstacked form

my_means.unstack()

key2,one,two
key1,,
a,-0.154507,0.509565
b,-1.454717,-0.337879


In [9]:
# when the grouping variables are in the same data frame as the variable used, we can skip the full name of the columns

df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.154507  0.309301
     two   0.509565  0.618821
b    one  -1.454717 -1.239709
     two  -0.337879  0.782384

## Getting Group Sizes

A handy way to get counts within groups


In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [11]:
df.groupby(['key1', 'key2']).size().unstack()

key2,one,two
key1,,
a,2,1
b,1,1


In [12]:
## Iterating over groups -

for name, group in df.groupby('key1'):
        print(name)
        print(group)


a
  key1 key2     data1     data2
0    a  one  0.902170  0.729726
1    a  two  0.509565  0.618821
4    a  one -1.211184 -0.111124
b
  key1 key2     data1     data2
2    b  one -1.454717 -1.239709
3    b  two -0.337879  0.782384


In [13]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.902170  0.729726
4    a  one -1.211184 -0.111124
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.509565  0.618821
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.454717 -1.239709
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.337879  0.782384


In [14]:
# we can take a groupby and convert it into a dictionary,  which is an interesing idea for look-ups

pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.454717,-1.239709
3,b,two,-0.337879,0.782384


In [15]:
pieces.keys()

dict_keys(['a', 'b'])

In [16]:
pieces.values()

dict_values([  key1 key2     data1     data2
0    a  one  0.902170  0.729726
1    a  two  0.509565  0.618821
4    a  one -1.211184 -0.111124,   key1 key2     data1     data2
2    b  one -1.454717 -1.239709
3    b  two -0.337879  0.782384])

In [17]:
pieces['a']

,key1,key2,data1,data2
0,a,one,0.902170,0.729726
1,a,two,0.509565,0.618821
4,a,one,-1.211184,-0.111124


This idea of putting a table into a dictionary for lookup purposes is really interesting.    I'm not sure just how I'd make use
of the ability to do this, but it's really an intriguing idea.

## Grouping using a Dictionary or a Series

In [18]:
people = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'],index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

# add some missing data points

people.iloc[2:3, [1, 2]] = np.nan

people

,a,b,c,d,e
Joe,0.980147,0.328641,-0.160397,0.551731,0.692649
Steve,0.511343,1.167967,-1.504613,-0.581568,0.100048
Wes,0.124514,NaN,NaN,-0.328108,-0.605308
Jim,0.265673,0.344851,0.601722,-0.265073,-0.241952
Travis,-2.050038,-0.178946,-1.962275,2.060543,-0.294307


In [19]:
# now suppose we want to map multiple columns to a single category,  such as red, blue, orange.   We can use a dictionary as a map of this correspondenc

mapping = {'a': 'red', 'b': 'red', 'c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}

In [20]:
by_column = people.groupby(mapping, axis=1)

In [21]:
by_column.sum()

,blue,red
Joe,0.391335,2.001437
Steve,-2.086180,1.779358
Wes,-0.328108,-0.480794
Jim,0.336649,0.368572
Travis,0.098268,-2.523291


At first glance, this looks a bit goofy,but we could use the dictionary to define expense or revenue categories and rapidly some over large datasets
using this tactic of mapping columns into subgroups and running all the calculations at once

In [22]:
# we can do this using a series as well

map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [23]:
people.groupby(map_series, axis=1).count()

# this does a nice job of locating those NA entries

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## Grouping with Functions

Any function passed as a group key will be called once per index value, with the return values being used as the group names

Suppose we wanted to group the people by the number of letters in their first name,   we could group by the length of the index

In [24]:
 people.groupby(len).sum()

,a,b,c,d,e
3,1.370334,0.673493,0.441325,-0.041450,-0.154611
5,0.511343,1.167967,-1.504613,-0.581568,0.100048
6,-2.050038,-0.178946,-1.962275,2.060543,-0.294307


In [25]:
# it is also possible to mix grouping by  arrays,  dictionaries, series and functions when grouping
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.124514  0.328641 -0.160397 -0.328108 -0.605308
  two  0.265673  0.344851  0.601722 -0.265073 -0.241952
5 one  0.511343  1.167967 -1.504613 -0.581568  0.100048
6 two -2.050038 -0.178946 -1.962275  2.060543 -0.294307

In [26]:
## Grouping with a Multiindex

In [27]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],[1, 3, 5, 1, 3]],names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.263344 -0.629650 -1.726117 -0.400244 -0.351359
1      1.651325 -1.606745 -0.406715  0.613252 -0.752462
2      1.357437  0.677094  0.684367 -0.084770 -1.835067
3     -1.593245  0.599841  0.340883 -1.012603 -0.326771

In [28]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [29]:
hier_df.groupby(level='tenor', axis=1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


This is an interesting way to rapidly summarize over a Multiindex

## Data Aggregration

Data Aggregration is any operation that summarises or extracts a single scalar value from an array

count, sum, mean, median, std, var, min, max, prod, first, last

In [30]:
# quantile can be used as well

df

,key1,key2,data1,data2
0,a,one,0.902170,0.729726
1,a,two,0.509565,0.618821
2,b,one,-1.454717,-1.239709
3,b,two,-0.337879,0.782384
4,a,one,-1.211184,-0.111124


In [31]:
grouped=df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.823649
b   -0.449563
Name: data1, dtype: float64

In [32]:
# creating your own aggregation functions

def peak_2_peak(x):
    return x.max()-x.min()

# used this in an agg() function

df.groupby('key2').agg(peak_2_peak)

<ipython-input-32-1ddc0c1955cd>:8: FutureWarning: ['key1'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df.groupby('key2').agg(peak_2_peak)


,data1,data2
key2,,
one,2.356887,1.969435
two,0.847444,0.163564


In [33]:
# describe is not truly an aggregation, but it does work with a groupby

df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.066851  1.124083 -1.211184 -0.350809  0.509565  0.705868   
b      2.0 -0.896298  0.789724 -1.454717 -1.175508 -0.896298 -0.617088   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.902170   3.0  0.412474  0.456827 -0.111124  0.253848  0.618821   
b    -0.337879   2.0 -0.228662  1.429836 -1.239709 -0.734186 -0.228662   

                          
           75%       max  
key1                      
a     0.674273  0.729726  
b     0.276861  0.782384

That's handy,   I hadn't seen that before

# Generalized split-apply_combine operations

the most general method is the apply() function,   similar to the R apply operation

In [34]:
# get the seaborn tips dataset

import seaborn as sns
tips = sns.load_dataset("tips")

In [35]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [36]:
tips['tip_pct']=tips.tip/tips.total_bill*100

In [37]:
# a function to select the top n values from a given column of a data frame,   n defaults to 5,  column to tip_pct
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips,6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,27.952481
183,23.17,6.50,Male,Yes,Sun,Dinner,4,28.053517
232,11.61,3.39,Male,No,Sat,Dinner,2,29.198966
67,3.07,1.00,Female,Yes,Sat,Dinner,1,32.573290
178,9.60,4.00,Female,Yes,Sun,Dinner,2,41.666667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,71.034483


In [38]:
top(tips,6,"total_bill")

,total_bill,tip,sex,smoker,day,time,size,tip_pct
102,44.30,2.50,Female,Yes,Sat,Dinner,3,5.643341
182,45.35,3.50,Male,Yes,Sun,Dinner,3,7.717751
156,48.17,5.00,Male,No,Sun,Dinner,6,10.379905
59,48.27,6.73,Male,No,Sat,Dinner,4,13.942407
212,48.33,9.00,Male,No,Sat,Dinner,4,18.621974
170,50.81,10.00,Male,Yes,Sat,Dinner,3,19.681165


In [39]:
# groupby smoker and apply this function- not that this is not an aggregation (why?)
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size    tip_pct
smoker                                                                    
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  27.952481
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  28.053517
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  32.573290
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  41.666667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  71.034483
No     88        24.71  5.85    Male     No  Thur   Lunch     2  23.674626
       185       20.69  5.00    Male     No   Sun  Dinner     5  24.166264
       51        10.29  2.60  Female     No   Sun  Dinner     2  25.267250
       149        7.51  2.00    Male     No  Thur   Lunch     2  26.631158
       232       11.61  3.39    Male     No   Sat  Dinner     2  29.198966

In [40]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
Yes    Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   
       Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
No     Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
       Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   

                   tip_pct  
smoker day                  
Yes    Thur 197  11.598237  
       Fri  95   11.774956  
       Sat  170  19.681165  
       Sun  182   7.717751  
No     Thur 142  12.138869  
       Fri  94   14.285714  
       Sat  212  18.621974  
       Sun  156  10.379905

In [41]:
# suppressing the group keys,  so the index is not show

tips.groupby('smoker', group_keys=False).apply(top)


,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,27.952481
183,23.17,6.50,Male,Yes,Sun,Dinner,4,28.053517
67,3.07,1.00,Female,Yes,Sat,Dinner,1,32.573290
178,9.60,4.00,Female,Yes,Sun,Dinner,2,41.666667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,71.034483
88,24.71,5.85,Male,No,Thur,Lunch,2,23.674626
185,20.69,5.00,Male,No,Sun,Dinner,5,24.166264
51,10.29,2.60,Female,No,Sun,Dinner,2,25.267250
149,7.51,2.00,Male,No,Thur,Lunch,2,26.631158
232,11.61,3.39,Male,No,Sat,Dinner,2,29.198966


## Quantile and Bucket Analysis

cut() and qcut() can be used for binning data,  we can use them to categorize ssamples within quintiles,  say characteristics of
customers by their profitability quintiles

Bucket analysis is divided records into "buckets" or groupings, to see what the characteristics within each bucket are

In [42]:
frame = pd.DataFrame({'data1': np.random.randn(1000),'data2': np.random.randn(1000)})

# these are equal length buckets

quartiles = pd.cut(frame.data1, 4)

quartiles[:10]

0    (-1.513, 0.403]
1      (0.403, 2.32]
2    (-1.513, 0.403]
3    (-1.513, 0.403]
4    (-1.513, 0.403]
5      (0.403, 2.32]
6      (0.403, 2.32]
7    (-1.513, 0.403]
8    (-1.513, 0.403]
9    (-1.513, 0.403]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.437, -1.513] < (-1.513, 0.403] < (0.403, 2.32] <
                                           (2.32, 4.236]]

In [43]:
# we'll make a function to calculate some basic statistics

def get_stats(group):
    return {'min': group.min(), 'max': group.max(),'count': group.count(), 'mean': group.mean()}

frame.data2.groupby(quartiles).apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.437, -1.513]",-2.306631,2.524232,63.0,0.049252
"(-1.513, 0.403]",-2.803468,2.678421,621.0,-0.029296
"(0.403, 2.32]",-3.144650,2.902658,300.0,-0.021375
"(2.32, 4.236]",-2.769468,1.453298,16.0,-0.521737


#Question:
    
    Modify the above to include a count column,  which should appear first.   Also include the range, which is the difference
            between min and max
    
    Then repeat the quartile analyis using qcut, instead of cut.    Cut gives equal size intervals, qcut gives equal counts

## Random Sampling and Permutation

In [44]:
#Build a data series to simulate or indicate a deck of cards

# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [45]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

# What is in the deck series?    How were these variables formed?

In [46]:
# lets create a function to draw 5 cards at random
# we are uisng the sample function

def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

3H    3
7D    7
2S    2
4C    4
5D    5
dtype: int64

In [47]:
# create a lambda function that returns the suit of the card, it just returns the last letter of the card name

get_suit = lambda card: card[-1] # last letter is suit

# this will give us two random cards per suit

deck.groupby(get_suit).apply(draw, n=2)

C  2C      2
   10C    10
D  2D      2
   JD     10
H  QH     10
   3H      3
S  5S      5
   6S      6
dtype: int64

In [48]:
# omitting the index of the group keys (C,D,H,S)
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)


AC     1
JC    10
8D     8
AD     1
9H     9
AH     1
QS    10
7S     7
dtype: int64

In [49]:
## Pivot Tables and Cross Tabulation in Pandas


In [50]:
# a pivot table of the tips data set,  broken out by day and smoker
tips.pivot_table(index=['day', 'smoker'])

<ipython-input-50-ec92957bbbd6>:2: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  tips.pivot_table(index=['day', 'smoker'])


size       tip    tip_pct  total_bill
day  smoker                                           
Thur Yes     2.352941  3.030000  16.386327   19.190588
     No      2.488889  2.673778  16.029808   17.113111
Fri  Yes     2.066667  2.714000  17.478305   16.813333
     No      2.250000  2.812500  15.165044   18.420000
Sat  Yes     2.476190  2.875476  14.790607   21.276667
     No      2.555556  3.102889  15.804766   19.661778
Sun  Yes     2.578947  3.516842  18.725032   24.120000
     No      2.929825  3.167895  16.011294   20.506667

In [51]:
# we can restrict the data in the table to just tip percentage and size
# and then index by time and day,  with columns by smoker

tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],columns='smoker')


# this is giving us a multiindex on both the columns and the rows.   This is an easy way to create a pretty complex table

size              tip_pct           
smoker            Yes        No        Yes         No
time   day                                           
Lunch  Thur  2.352941  2.500000  16.386327  16.031067
       Fri   1.833333  3.000000  18.893659  18.773467
Dinner Thur       NaN  2.000000        NaN  15.974441
       Fri   2.222222  2.000000  16.534736  13.962237
       Sat   2.476190  2.555556  14.790607  15.804766
       Sun   2.578947  2.929825  18.725032  16.011294

In [52]:
# we can add row and column marginal values,  that is to say row and column-wise totals

tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],columns='smoker', margins=True)

size                        tip_pct                      
smoker            Yes        No       All        Yes         No        All
time   day                                                                
Lunch  Thur  2.352941  2.500000  2.459016  16.386327  16.031067  16.130074
       Fri   1.833333  3.000000  2.000000  18.893659  18.773467  18.876489
Dinner Thur       NaN  2.000000  2.000000        NaN  15.974441  15.974441
       Fri   2.222222  2.000000  2.166667  16.534736  13.962237  15.891611
       Sat   2.476190  2.555556  2.517241  14.790607  15.804766  15.315172
       Sun   2.578947  2.929825  2.842105  18.725032  16.011294  16.689729
All          2.408602  2.668874  2.569672  16.319604  15.932846  16.080258

In [53]:
# if we pass in an aggregation function, we can get different types of information in the pivot table
# passing in aggfunc=len should give us a cross table

tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],columns='smoker', margins=True,aggfunc=len)

size             tip_pct            
smoker        Yes     No  All     Yes     No  All
time   day                                       
Lunch  Thur  17.0   44.0   61    17.0   44.0   61
       Fri    6.0    1.0    7     6.0    1.0    7
Dinner Thur   NaN    1.0    1     NaN    1.0    1
       Fri    9.0    3.0   12     9.0    3.0   12
       Sat   42.0   45.0   87    42.0   45.0   87
       Sun   19.0   57.0   76    19.0   57.0   76
All          93.0  151.0  244    93.0  151.0  244

In [54]:
# we can pass in a fill value to replace missing data

tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],columns='smoker', margins=True,aggfunc=len,fill_value=0)

size           tip_pct          
smoker       Yes   No  All     Yes   No  All
time   day                                  
Lunch  Thur   17   44   61      17   44   61
       Fri     6    1    7       6    1    7
Dinner Thur    0    1    1       0    1    1
       Fri     9    3   12       9    3   12
       Sat    42   45   87      42   45   87
       Sun    19   57   76      19   57   76
All           93  151  244      93  151  244

In [55]:
# Table of Max values

tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],columns='smoker', margins=True,aggfunc=max,fill_value=0)

size           tip_pct                      
smoker       Yes No All        Yes         No        All
time   day                                              
Lunch  Thur    4  6   6  24.125452  26.631158  26.631158
       Fri     2  3   3  25.931446  18.773467  25.931446
Dinner Thur    0  2   2   0.000000  15.974441  15.974441
       Fri     4  2   4  26.348039  15.562472  26.348039
       Sat     5  4   5  32.573290  29.198966  32.573290
       Sun     5  6   6  71.034483  25.267250  71.034483
All            5  6   6  71.034483  29.198966  71.034483